# HPC intro

## Using Python

Python is of course a very useful programming language for data processing, analysis and visualization.

There are many tutorials and courses that will teach you Python, so that is not the scope of this tutorial.  Here you will learn how to run Python on our HPC infrastructure, assuming you are already familiar with the language and packages.

### Python scipts

Given that Python scripts are simple text files, you can create or modify them using your favorite editor.  You can do this for instance on the infrastructure using `nano`, or on your own system and transfer the finished script or module to the HPC system.

To build up gradually, you can start with a very simple script that takes a string as a command line argument, and prints a greeting to standard output.  Your script is stored in a file `hello.py` which could look like this.

```
#!/usr/bin/env python

import argparse


arg_parser = argparse.ArgumentParser(description='say hello')
arg_parser.add_argument('name', help='who to say hello to')
options = arg_parser.parse_args()

print('Hello ' + options.name + '!')
```

The only module used in this script, `argparse` is in Python's standard library, and the script has been written in such a way that it will work with any version of Python.  In practice, use f-strings, and a recent version of Python.

### Running simple scripts

You can run it by giving it as a command line argument to the Python interpreter.

In [1]:
python hello.py there

Hello there!


You can of course easily check which version of Python is used to run your script, as well as where it is installed on the system.

In [2]:
python --version

Python 3.10.8


In [3]:
which python

/home/gjb/mambaforge/bin/python


However, often the version of the Python interpreter that comes with the operating system is not the one you would like to use, or you want to use Python packages that are not installed, so what can you do?

Here, we will assume that you have a (fairly simple) script that computes a function for an array of floating point values, and that writes a line plot that shows the results to a file.

```
#!/usr/bin/env python

import matplotlib.pyplot as plt
import numpy as np


x = np.linspace(-2*np.pi, 2*np.pi, 501)
y = np.sin(x)

plt.plot(x, y, '-')
plt.savefig('sin.png')
```

This script requires both the numpy and matplotlib packages, running it with the default Python interpreter is not going to be a big success.

In [6]:
python 002_artefacts/sin_plot.py

Traceback (most recent call last):
  File "/home/gjb/Projects/Personas-material/tutorials/basics/002_artefacts/sin_plot.py", line 3, in <module>
    import numpy as np
ModuleNotFoundError: No module named 'numpy'


: 1

As it happens, there are quite a number of options,

  1. install packages in your home directory's `.local` directory using `pip`;
     * advantages: fairly straightforward
     * disadvantages: sure to create a dependency mess later on, performance is likely to be an issue
     * conclusion: *please don't*
  1. use the module system and Python versions and packages installed by your system administrator;
     * advantages: typically excellent performance
     * disadvantages: since system administrators really can't install any and all Python packages, you
       may have to install some packages yourself anyway
     * conclusion: perfect if you have no requirements beyond the packages that are available
  1. use a package manager such as [miniconda](https://docs.conda.io/en/latest/miniconda.html)
     or [mamba](https://github.com/mamba-org/mamba)
     * advantages: you have full control over the versions of Python and all packages
     * disadvantages: unless you know what you are doing, performance may be an issue
     * conclusion: way to go if you know what you are doing
  1. use [apptainer](https://apptainer.org/) or [podman](https://podman.io/) containers
     * advantages: if you know what you are doing, you can create a reproducible environment that is
       portable across systems
     * disadvantages: more involved than the other approaches, with considerable pitfalls
     * conclusion: not for the faint of heart
    
Given that the first option is not recommended at all, and the fourth goes beyond the scope of this tutorial, you will learn how to

  * use the module system
  * install and use miniconda

### Software module system

### Package manager: miniconda

## Summary

## Where to go from here?